<a href="https://colab.research.google.com/github/oscarmejias/deeplearning/blob/master/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Aprendizaje por transferencia**

##**importamos las librerias necesarias**

In [1]:
import os
import zipfile
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.2.0-rc2'

In [2]:
!wget --no-check-certificate \
  http://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2020-04-09 23:24:17--  http://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   167MB/s    in 0.4s    

2020-04-09 23:24:18 (167 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



##**Descomprimimos el dataset de Perros y Gatos**

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

**Configurar la ruta al dataset**

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## **Construir el modelo**

**Cargamos el modelo preentrenado (MobileNetV2)**

In [0]:
IMG_SHAPE = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top= False, weights= "imagenet")

In [11]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

**Congelar el modelo base**

In [0]:
base_model.trainable = False

**Definir la cabezera personalizada para nuestra red neuronal**

In [13]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [15]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units= 1, activation= "sigmoid")(global_average_layer)

In [0]:
model = tf.keras.models.Model(inputs= base_model.input, outputs= prediction_layer)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

**Compilamos el modelo**

In [0]:
model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss= "binary_crossentropy", metrics=["accuracy"])

##**Crear generadores de datos**

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_val = ImageDataGenerator(rescale=1/255.)

In [21]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size= (128,128), batch_size= 128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [22]:
val_generator = data_gen_val.flow_from_directory(validation_dir, target_size= (128,128), batch_size= 128, class_mode="binary")

Found 1000 images belonging to 2 classes.


**Entrenar el modelo**

In [0]:
model.fit(train_generator, epochs= 5, validation_data= val_generator)

In [0]:
val_loss, val_acc = model.evaluate(val_generator)

In [0]:
print("Validation acuraccy after transfer learning: {}".format(val_acc))

#**Puesta a punto de parametros**

**Descongelamos el modelo**

In [0]:
base_model.trainable = True

In [0]:
print("Number of layers in de base model: {}".format(len(base_model.layers)))

In [0]:
fine_tune_at = 155

In [0]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

**Compilar el modelo para la puesta a punto**

In [0]:
model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss= "binary_crossentropy",
              metrics= ["accuracy"])

**Puesta a punto**

In [0]:
model.fit(train_generator,
          epoch= 5,
          batch_size= 32,
          validation_data= val_generator)